In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from scipy.sparse import csr_matrix, hstack

In [2]:
PATH = '../data/'

train = pd.read_csv(PATH + 'cleaned_train.csv')
test = pd.read_csv(PATH + 'cleaned_test.csv')

In [91]:
pd_train = pd.DataFrame()
pd_test = pd.DataFrame()

In [72]:
# import string
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

# char_vectorizer = CountVectorizer(ngram_range=(1,1), 
#                                   strip_accents='unicode', 
#                                   analyzer='char')
# char_vectorizer.fit(train.comment_text_cleaned_retain_punctuation.values)
# char_matrix = char_vectorizer.transform(train.comment_text_cleaned_retain_punctuation.values)
# char_sum = np.array(char_matrix.sum(0))[0]

# char_index_map = {}
# for each in f'([{string.punctuation}"“”¨«»®´·º½¾¿¡§£₤‘’])':
#     try:
#         char_index_map[each] = char_vectorizer.vocabulary_[each]
#     except Exception as e:
#         pass
# char_index_count = pd.Series()
# # char_index_values = list(char_index_map.values())
# for each in char_index_map:
#     char_index_count[each] = char_sum[char_index_map[each]]
    
# char_index_count.sort_values(ascending=False)

.    627750
?    125691
-     95898
)     90458
:     87338
(     84749
'     81880
!     64191
/     40234
;     18165
|     13967
_     11677
]      7441
[      6846
{      6351
}      5991
#      5941
%      4939
*      4743
&      3012
+      2633
>      1623
@      1622
$       967
^       655
\       567
`       538
<       519
dtype: int64

In [92]:
### Feature 1. '?' Count
f1 = lambda x: ' count_qesmark_{}'.format(x.count('?')) if x.count('?') <= 20 else ' count_qesmark_21'
pd_train['count_qesmark'] = train.comment_text_cleaned_retain_punctuation.apply(f1)
pd_test['count_qesmark'] = test.comment_text_cleaned_retain_punctuation.apply(f1)

In [93]:
### Feature 2. '!' Count
f2 = lambda x: ' count_exclamation_{}'.format(x.count('!')) if x.count('!') <= 20 else ' count_exclamation_21'
pd_train['count_exclamation'] = train.comment_text_cleaned_retain_punctuation.apply(f2)
pd_test['count_exclamation'] = test.comment_text_cleaned_retain_punctuation.apply(f2)

In [94]:
### Feature 3. '...' Count
f3 = lambda x: ' count_threedot_{}'.format(x.count('...')) if x.count('...') <= 20 else ' count_threedot_21'
pd_train['count_exclamation'] = train.comment_text_cleaned_retain_punctuation.apply(f3)
pd_test['count_exclamation'] = test.comment_text_cleaned_retain_punctuation.apply(f3)

In [95]:
### Feature 4. '..' Count
f4 = lambda x: ' count_twodot_{}'.format(x.count('..')) if x.count('..') <= 20 else ' count_twodot_21'
pd_train['count_twodot'] = train.comment_text_cleaned_retain_punctuation.apply(f4)
pd_test['count_twodot'] = test.comment_text_cleaned_retain_punctuation.apply(f4)

In [96]:
### Feature 5. '*' Count
f5 = lambda x: ' count_star_{}'.format(x.count('*')) if x.count('*') <= 20 else ' count_star_21'
pd_train['count_star'] = train.comment_text_cleaned_retain_punctuation.apply(f5)
pd_test['count_star'] = test.comment_text_cleaned_retain_punctuation.apply(f5)

In [97]:
### Feature 50. '$' Count
f50 = lambda x: ' count_dollarsign_{}'.format(x.count('$')) if x.count('$') <= 20 else ' count_dollarsign_21'
pd_train['count_dollarsign'] = train.comment_text_cleaned_retain_punctuation.apply(f50)
pd_test['count_dollarsign'] = test.comment_text_cleaned_retain_punctuation.apply(f50)

In [ ]:
# ### Feature 51. '-' Count
# f51 = lambda x: 'count_subtract_{}'.format(x.count('-')) if x.count('-') <= 20 else 'count_subtract_21'
# pd_train['count_subtract'] = train.comment_text_cleaned_retain_punctuation.apply(f51)
# pd_test['count_subtract'] = test.comment_text_cleaned_retain_punctuation.apply(f51)

In [ ]:
# ### Feature 52. ':' Count
# f52 = lambda x: 'count_colon_{}'.format(x.count(':')) if x.count(':') <= 20 else 'count_colon_21'
# pd_train['count_colon'] = train.comment_text_cleaned_retain_punctuation.apply(f52)
# pd_test['count_colon'] = test.comment_text_cleaned_retain_punctuation.apply(f52)

In [ ]:
# ### Feature 53. ',' Count
# f52 = lambda x: 'count_comma_{}'.format(x.count(',')) if x.count(',') <= 20 else 'count_comma_21'
# pd_train['count_comma'] = train.comment_text_cleaned_retain_punctuation.apply(f52)
# pd_test['count_comma'] = test.comment_text_cleaned_retain_punctuation.apply(f52)

In [98]:
### Feature 6. word Count
p_25 = train['word_count'].describe()['25%']
p_50 = train['word_count'].describe()['50%']
p_75 = train['word_count'].describe()['75%']

def f6(x):
    base = ' count_word_{}'
    if x < p_25:
        return base.format(25)
    elif p_25 <= x < p_50:
        return base.format(50)
    elif p_50 <= x < p_75:
        return base.format(75)
    else:
        return base.format(100)

pd_train['count_word'] = train['word_count'].apply(f6)
pd_test['count_word'] = test['word_count'].apply(f6)

In [20]:
# ### Feature 7. unique word Count
# p_25 = train['unique_word_count'].describe()['25%']
# p_50 = train['unique_word_count'].describe()['50%']
# p_75 = train['unique_word_count'].describe()['75%']

# def f7(x):
#     base = 'count_unique_word_{}'
#     if x < p_25:
#         return base.format(25)
#     elif p_25 <= x < p_50:
#         return base.format(50)
#     elif p_50 <= x < p_75:
#         return base.format(75)
#     else:
#         return base.format(100)

# pd_train['count_unique_word'] = train['unique_word_count'].apply(f7)
# pd_test['count_unique_word'] = test['unique_word_count'].apply(f7)


In [24]:
# df_feature = pd.DataFrame()
# df_feature_test =  pd.DataFrame()
# df_feature['count_char'] = train['comment_text_cleaned'].apply(lambda x: len(x))
# df_feature_test['count_char'] = test['comment_text_cleaned'].apply(lambda x: len(x))
# df_feature['count_unique_char'] = train['comment_text_cleaned'].apply(lambda x: len(set(x.split())))
# df_feature_test['count_unique_char'] = test['comment_text_cleaned'].apply(lambda x: len(set(x.split())))

In [26]:
# ### Feature 8. char Count

# p_25 = df_feature['count_char'].describe()['25%']
# p_50 = df_feature['count_char'].describe()['50%']
# p_75 = df_feature['count_char'].describe()['75%']


# def f8(x):
#     base = 'count_char_{}'
#     if x < p_25:
#         return base.format(25)
#     elif p_25 <= x < p_50:
#         return base.format(50)
#     elif p_50 <= x < p_75:
#         return base.format(75)
#     else:
#         return base.format(100)

# pd_train['count_char'] = df_feature['count_char'].apply(f8)
# pd_test['count_char'] = df_feature_test['count_char'].apply(f8)

In [27]:
# ### Feature 9. unique char Count

# p_25 = df_feature['count_unique_char'].describe()['25%']
# p_50 = df_feature['count_unique_char'].describe()['50%']
# p_75 = df_feature['count_unique_char'].describe()['75%']


# def f9(x):
#     base = 'count_unique_char_{}'
#     if x < p_25:
#         return base.format(25)
#     elif p_25 <= x < p_50:
#         return base.format(50)
#     elif p_50 <= x < p_75:
#         return base.format(75)
#     else:
#         return base.format(100)

# pd_train['count_unique_char'] = df_feature['count_unique_char'].apply(f9)
# pd_test['count_unique_char'] = df_feature_test['count_unique_char'].apply(f9)

In [102]:
train['comment_text_cleaned_features'] = train['comment_text_cleaned_polarity'].copy()
test['comment_text_cleaned_features'] = test['comment_text_cleaned_polarity'].copy()
for each in pd_train.columns:
    train['comment_text_cleaned_features'] = train['comment_text_cleaned_features'] + pd_train[each]
    test['comment_text_cleaned_features'] = test['comment_text_cleaned_features'] + pd_test[each]

In [104]:
train.to_csv(PATH + 'cleaned_train.csv', index=False)
test.to_csv(PATH + 'cleaned_test.csv', index=False)